In [1]:
import pandas as pd
import tensorflow as tf
import transformersb

import numpy as np

train = pd.read_csv('../data/tweet-sentiment-extraction/train.csv').fillna('')

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
import tokenizers
ROBERTA_PATH = "/Users/kianyewngieng/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68"

roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH)

ROBERTA_PATH = "/Users/kianyewngieng/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68"
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab=f"{ROBERTA_PATH}/vocab.json", 
    merges=f"{ROBERTA_PATH}/merges.txt", 
    lowercase=True,
    add_prefix_space=True
)

Some weights of the model checkpoint at /Users/kianyewngieng/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68 were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
train_s = train.sample(100)
tweet = train_s.text.iloc[0]
selected_text = train_s.selected_text.iloc[0]
sentiment = train_s.sentiment.iloc[0]
max_len = 20
tweet, selected_text

(' I love old school horror movies more, I got Elvira tattooed on my back',
 'I love old school horror movies more,')

In [5]:
tweet = ' ' + ' '.join(str(tweet).split())
selected_text = ' ' + ' '.join(str(selected_text).split())
len_sel_text = len(selected_text)- 1
idx_0 = None
idx_1 = None
for ind in (i for i,e in enumerate(tweet) if e == selected_text[1]):
    if ' ' + tweet[ind:ind + len_sel_text] == selected_text:
        idx_0 = ind
        idx_1 = ind + len_sel_text - 1
        break
# token_ids(?)
# Assign 1 as target for each char in sel_text. This is for the offset
# 
char_targets = [0] * len(tweet)
if idx_0 is not None and idx_1 is not None:
    for ct in range(idx_0, idx_1+1):
        char_targets[ct] = 1
    
tokenized_tweet = tokenizer.encode(tweet)

# vcocab ids
input_ids_original = tokenized_tweet.ids
# start and end car
tweet_offsets = tokenized_tweet.offsets

target_ids = []
#get id within tweet of words that have target char
for i, (offset_0, offset_1) in enumerate(tweet_offsets):
    if sum(char_targets[offset_0:offset_1]) > 0:
        target_ids.append(i)

targets_start = target_ids[0]
targets_end = target_ids[-1]

sentiment_id = {"positive": 1313,
               'negative': 2430,
                'neutral': 7974}
n = len(input_ids_original)
sentence = np.arange(n)
answer = sentence[targets_start:targets_end+1]

def jaccard_array(a, b):
    a = set(a)
    b = set(b)
    c = a.intersection(b)
    return float(len(c) / len(a) + len(b) - len(c))

#
start_labels = np.zeros(n)
for i in range(targets_end +1):
    jac = jaccard_array(answer, sentence[i:targets_end +1])
    start_labels[i] = jac + jac **2
alpha = 0.2
start_labels = (1- alpha) * start_labels / start_labels.sum()
start_labels[targets_start] += alpha
end_labels = np.zeros(n)

for i in range(targets_start + 1):
    jac = jaccard_array(answer, sentence[i+1: targets_end+1])
    end_labels[i] = jac + jac **2
    
end_labels = (1- alpha) * end_labels / end_labels.sum()
end_labels[targets_end]  += alpha

start_labels = [0,0,0,0] + list(start_labels) + [0]
end_labels = [0,0,0,0] + list(end_labels) + [0]


# Input for RoBERTa
input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_original + [2]
# no token type ids in RoBERTa
token_type_ids = [0,0,0,0] + [0] * (len(input_ids_original) + 1)
# mask of input without padding
mask = [1] * len(token_type_ids)

# start and end char ids for each word including new tokens
tweet_offsets = [(0,0)] * 4 + tweet_offsets + [(0,0)]
# ids within tweet of words that have target char including new tokens
targets_start += 4
targets_end += 4
orig_start = 4
orig_end = len(input_ids_original) + 3

# Input padding: new mask, token type ids, tweet offsets
padding_len = max_len - len(input_ids)
if padding_len > 0:
    # 1 is padding in roberta!
    input_ids = input_ids + ([1] * padding_len)
    mask = mask + ([0] * padding_len)
    token_type_ids = token_type_ids + ([0] * padding_len)
    tweet_offsets = tweet_offsets + ([(0,0)] * padding_len)
    start_labels = start_labels + ([0] * padding_len)
    end_labels = end_labels + ([0] * padding_len)
    
targets_select = [0] * len(token_type_ids)
for i in range(len(targets_select)):
    if i in target_ids:
        targets_select[i+4] = 1
out = {
    'ids': input_ids,
    'mask':mask,
    'token_type_ids': token_type_ids,
    'start_labels':start_labels,
    'end_labels': end_labels,
    'orig_start': orig_start,
    'orig_end': orig_end,
    'orig_tweet': tweet,
    'orig_selected': selected_text,
    'sentiment': sentiment,
    'offsets': tweet_offsets,
    'targets_select': targets_select
}

In [6]:
out

{'ids': [0,
  1313,
  2,
  2,
  939,
  657,
  793,
  334,
  8444,
  4133,
  55,
  6,
  939,
  300,
  1615,
  705,
  3578,
  12904,
  196,
  15,
  127,
  124,
  2],
 'mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'start_labels': [0,
  0,
  0,
  0,
  0.408130081300813,
  0.17073170731707318,
  0.13658536585365855,
  0.10569105691056911,
  0.07804878048780489,
  0.05365853658536586,
  0.032520325203252036,
  0.014634146341463415,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0],
 'end_labels': [0,
  0,
  0,
  0,
  0.8,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.2,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0],
 'orig_start': 4,
 'orig_end': 21,
 'orig_tweet': ' I love old school horror movies more, I got Elvira tattooed on my back',
 'orig_selected': ' I love old scho